# Build a CNN for image recognition.

### Name: Zhili Yu

## 1. Data preparation

### 1.1. Load data


In [1]:
from keras.datasets import cifar10
import numpy

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
mean = numpy.mean(x_train,axis=(0,1,2,3))
std = numpy.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print('shape of x_train: ' + str(x_train.shape))
print('shape of y_train: ' + str(y_train.shape))
print('shape of x_test: ' + str(x_test.shape))
print('shape of y_test: ' + str(y_test.shape))
print('number of classes: ' + str(numpy.max(y_train) - numpy.min(y_train) + 1))

Using TensorFlow backend.


shape of x_train: (50000, 32, 32, 3)
shape of y_train: (50000, 1)
shape of x_test: (10000, 32, 32, 3)
shape of y_test: (10000, 1)
number of classes: 10


### 1.2. One-hot encode the labels

In the input, a label is a scalar in $\{0, 1, \cdots , 9\}$. One-hot encode transform such a scalar to a $10$-dim vector. E.g., a scalar ```y_train[j]=3``` is transformed to the vector ```y_train_vec[j]=[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]```.

1. Define a function ```to_one_hot``` that transforms an $n\times 1$ array to a $n\times 10$ matrix.

2. Apply the function to ```y_train``` and ```y_test```.

In [2]:
def to_one_hot(y, num_class=10):
    temp = numpy.zeros(shape=(y.shape[0],num_class))
    for i in range(y.shape[0]):
        temp[i,y[i]] = 1
    return temp

y_train_vec = to_one_hot(y_train)
y_test_vec = to_one_hot(y_test)

print('Shape of y_train_vec: ' + str(y_train_vec.shape))
print('Shape of y_test_vec: ' + str(y_test_vec.shape))

print(y_train[0])
print(y_train_vec[0])

Shape of y_train_vec: (50000, 10)
Shape of y_test_vec: (10000, 10)
[6]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


#### Remark: the outputs should be
* Shape of y_train_vec: (50000, 10)
* Shape of y_test_vec: (10000, 10)
* [6]
* [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]

### 1.3. Randomly partition the training set to training and validation sets

Randomly partition the 50K training samples to 2 sets:
* a training set containing 40K samples
* a validation set containing 10K samples


In [3]:
rand_indices = numpy.random.permutation(50000)
train_indices = rand_indices[0:40000]
valid_indices = rand_indices[40000:50000]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

x_tr = x_train[train_indices, :]
y_tr = y_train_vec[train_indices, :]


print('Shape of x_tr: ' + str(x_tr.shape))
print('Shape of y_tr: ' + str(y_tr.shape))
print('Shape of x_val: ' + str(x_val.shape))
print('Shape of y_val: ' + str(y_val.shape))

Shape of x_tr: (40000, 32, 32, 3)
Shape of y_tr: (40000, 10)
Shape of x_val: (10000, 32, 32, 3)
Shape of y_val: (10000, 10)


## 2. Build a CNN and tune its hyper-parameters

1. Build a convolutional neural network model
2. Use the validation data to tune the hyper-parameters (e.g., network structure, and optimization algorithm)
3. Try to achieve a validation accuracy as high as possible.

In [67]:
from keras.layers import GlobalAveragePooling2D ,GlobalMaxPooling2D, ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input, Add
from keras.models import Sequential, Model

def resnet_identity_block(x,filters,kernel_size,triger = 0,triger_d=0,rate=0.2):
    x_skip = x
    if triger_d == 1:
        x = Dropout(rate)(x)
    x = Conv2D(filters=filters,kernel_size=(kernel_size,kernel_size),padding='same')(x)
    if triger == 1:
        x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    return x

def resnet_cov_block(x,filters,kernel_size,triger = 0,triger_d=0,rate=0.2):
    x_skip = Conv2D(filters=filters,kernel_size=(kernel_size,kernel_size),padding='same')(x)
    if triger == 1:
        x_skip = BatchNormalization(axis=3)(x_skip)
    x_skip = Activation('relu')(x_skip)
    if triger_d == 1:
        x = Dropout(rate)(x)
    x = Conv2D(filters=filters,kernel_size=(kernel_size,kernel_size),padding='same')(x)
    if triger == 1:
        x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    return x

def auxiliary_output(x):
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(200)(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(200)(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(10, activation='softmax')(x)
    return x

def resnet(shape = (32,32,3)):
    x_input = Input(shape)
    x = x_input
    x = Conv2D(32,(3,3),padding='same',input_shape=shape, name="input_layor")(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = resnet_identity_block(x,32,3,0)
    x = resnet_identity_block(x,32,3,0)
    x = resnet_identity_block(x,32,3,0)
    x = resnet_identity_block(x,32,3,0)
    x = MaxPooling2D(2,2)(x)
    x = resnet_cov_block(x,64,3,0,1,0.5)
    x = resnet_identity_block(x,64,3,0,1,0.5)
    x = resnet_identity_block(x,64,3,0,1,0.5)
    x = resnet_identity_block(x,64,3,0,1,0.5)
    x = resnet_identity_block(x,64,3,0,1,0.5)
    x = resnet_identity_block(x,64,3,0,1,0.5)
    aux_output_0 = auxiliary_output(x)
    x = MaxPooling2D(2,2)(x)
    x = resnet_cov_block(x,128,3,1,1,0.5)
    x = resnet_identity_block(x,128,3,1,1,0.5)
    x = resnet_identity_block(x,128,3,1,1,0.5)
    x = resnet_identity_block(x,128,3,1,1,0.5)
    x = resnet_identity_block(x,128,3,1,1,0.5)
    x = resnet_identity_block(x,128,3,1,1,0.5)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1000)(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1000)(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    final = Dense(10, activation='softmax')(x)
    model = Model(inputs = x_input, outputs = (aux_output_0,final))
    return model

model = resnet((32,32,3))
model.summary()

# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32, 32, 3)))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3), padding= 'same'))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3), padding= 'same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), padding= 'same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3), padding= 'same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.3))
# model.add(Conv2D(128, (3, 3), padding= 'same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(128, (3, 3), padding= 'same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(GlobalMaxPooling2D())
# model.add(Dropout(0.3))
# model.add(Dense(1000))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(500))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dense(10, activation='softmax'))
# 
# model.summary()


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
input_layor (Conv2D)            (None, 32, 32, 32)   896         input_19[0][0]                   
__________________________________________________________________________________________________
batch_normalization_131 (BatchN (None, 32, 32, 32)   128         input_layor[0][0]                
__________________________________________________________________________________________________
activation_654 (Activation)     (None, 32, 32, 32)   0           batch_normalization_131[0][0]    
___________________________________________________________________________________________

In [68]:
from keras import optimizers

learning_rate = 3E-4 # to be tuned!

model.compile(loss=['categorical_crossentropy','categorical_crossentropy'],
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

In [69]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_tr)


In [70]:
def batch_generator(x,y,batch_size):
    step = 0
    while True:
        if step == x.shape[0]:
            step = 0
        x_temp = numpy.zeros(shape=(batch_size,x.shape[1],x.shape[2],x.shape[3]))
        y1_temp = numpy.zeros(shape=(batch_size,y.shape[1]))
        y2_temp = numpy.zeros(shape=(batch_size,y.shape[1]))
        for i in range(batch_size):
            x_temp[i] = x[step]
            y1_temp[i] = y[step]
            y2_temp[i] = y[step]
            step+=1
        yield(x_temp,[y1_temp,y2_temp])

In [ ]:
# from keras.callbacks.callbacks import EarlyStopping
# callback = EarlyStopping(monitor='val_acc',mode='max', verbose=1, min_delta=1E-8, patience=10, restore_best_weights=True)

batch_size = 400
history = model.fit_generator(batch_generator(x_tr,y_tr,batch_size=batch_size), steps_per_epoch=x_tr.shape[0]//batch_size,epochs=200, verbose=2, validation_data=(x_val, [y_val,y_val]))
# history = model.fit(x_tr, [y_tr,y_tr], batch_size=batch_size, epochs= 300, verbose=2, validation_data=(x_val, [y_val,y_val]))
# history = model.fit_generator(datagen.flow(x_tr, y_tr, batch_size=batch_size), steps_per_epoch=x_tr.shape[0]//batch_size,epochs=200, verbose=2, validation_data=(x_val, y_val))
# history = model.fit(x_tr, y_tr, batch_size=batch_size, epochs=50, verbose=2, validation_data=(x_val, y_val))

Epoch 1/200
 - 15s - loss: 5.2792 - dense_105_loss: 2.9073 - dense_108_loss: 2.3719 - dense_105_acc: 0.1304 - dense_108_acc: 0.1701 - val_loss: 5.5248 - val_dense_105_loss: 2.2253 - val_dense_108_loss: 3.2996 - val_dense_105_acc: 0.1711 - val_dense_108_acc: 0.1008
Epoch 2/200
 - 12s - loss: 3.9483 - dense_105_loss: 2.0488 - dense_108_loss: 1.8995 - dense_105_acc: 0.2124 - dense_108_acc: 0.2535 - val_loss: 5.4108 - val_dense_105_loss: 2.0763 - val_dense_108_loss: 3.3345 - val_dense_105_acc: 0.2269 - val_dense_108_acc: 0.1089
Epoch 3/200
 - 12s - loss: 3.6681 - dense_105_loss: 1.9225 - dense_108_loss: 1.7457 - dense_105_acc: 0.2547 - dense_108_acc: 0.3038 - val_loss: 4.2993 - val_dense_105_loss: 1.9005 - val_dense_108_loss: 2.3987 - val_dense_105_acc: 0.3387 - val_dense_108_acc: 0.1873
Epoch 4/200
 - 12s - loss: 3.4821 - dense_105_loss: 1.8281 - dense_108_loss: 1.6540 - dense_105_acc: 0.2914 - dense_108_acc: 0.3445 - val_loss: 3.7519 - val_dense_105_loss: 1.7313 - val_dense_108_loss: 2.0

In [59]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['dense_84_acc']
val_acc = history.history['val_dense_84_acc']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

KeyError: 'dense_84_acc'

### 3.1. Train the model on the entire training set

Why? Previously, you used 40K samples for training; you wasted 10K samples for the sake of hyper-parameter tuning. Now you already know the hyper-parameters, so why not using all the 50K samples for training?

In [90]:
from keras import optimizers

learning_rate = 3E-4 # to be tuned!

model.compile(loss=['categorical_crossentropy','categorical_crossentropy'],
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

In [91]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [94]:
batch_size = 200
# history = model.fit_generator(datagen.flow(x_train, y_train_vec, batch_size=batch_size), steps_per_epoch=x_train.shape[0]//batch_size,epochs=200, verbose=2)
history = model.fit_generator(batch_generator(x_train,y_train_vec,batch_size=batch_size), steps_per_epoch=x_train.shape[0]//batch_size,epochs=200, verbose=2)

Epoch 1/200
 - 19s - loss: 4.1086 - dense_87_loss: 2.0673 - dense_90_loss: 2.0412 - dense_87_acc: 0.2172 - dense_90_acc: 0.2284
Epoch 2/200
 - 17s - loss: 3.4158 - dense_87_loss: 1.7408 - dense_90_loss: 1.6750 - dense_87_acc: 0.3329 - dense_90_acc: 0.3634
Epoch 3/200
 - 17s - loss: 3.1232 - dense_87_loss: 1.6113 - dense_90_loss: 1.5119 - dense_87_acc: 0.3890 - dense_90_acc: 0.4339
Epoch 4/200
 - 17s - loss: 2.9093 - dense_87_loss: 1.5147 - dense_90_loss: 1.3946 - dense_87_acc: 0.4286 - dense_90_acc: 0.4855
Epoch 5/200
 - 17s - loss: 2.7255 - dense_87_loss: 1.4278 - dense_90_loss: 1.2977 - dense_87_acc: 0.4698 - dense_90_acc: 0.5254
Epoch 6/200
 - 17s - loss: 2.5711 - dense_87_loss: 1.3579 - dense_90_loss: 1.2131 - dense_87_acc: 0.4973 - dense_90_acc: 0.5619
Epoch 7/200
 - 17s - loss: 2.4456 - dense_87_loss: 1.2950 - dense_90_loss: 1.1506 - dense_87_acc: 0.5241 - dense_90_acc: 0.5838
Epoch 8/200
 - 17s - loss: 2.3419 - dense_87_loss: 1.2504 - dense_90_loss: 1.0915 - dense_87_acc: 0.5422

### 3.2. Evaluate the model on the test set

In [96]:
loss_and_acc = model.evaluate(x_test, [y_test_vec,y_test_vec])
print('loss = ' + str(loss_and_acc[1]))
print('accuracy = ' + str(loss_and_acc[3]))

10000/10000 [==============================] - 3s 312us/step
loss = 0.5042510032653809
accuracy = 0.833899974822998


In [67]:
!jupyter nbconvert --to html HM4.ipynb

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern 'HM4.ipynb' matched no files
